In [1]:
import json

def profile_to_natural_text(user):
    lines = []
    lines.append(f"Name: {user['name']}")
    lines.append(f"Age: {user['age']}")
    lines.append(f"Gender: {user.get('gender','')}")
    lines.append(f"Location: {user.get('location','')}, {user.get('country','')}")
    lines.append(f"Education: {user.get('education','')}")
    lines.append(f"Occupation: {user.get('occupation','')}")
    lines.append(f"Marital status: {user.get('marital_status','')}")
    lines.append(f"Learning style: {user.get('learning_style','')}")
    lines.append(f"Humor style: {user.get('humor_style','')}")
    lines.append(f"Interests: {', '.join(user['interests'])}")
    lines.append(f"Languages: {', '.join(user['languages'])}")
    if 'bio' in user:
        lines.append(f"Bio: {user['bio']}")
    # Add personality traits summary (list top 3 by value)
    if 'personality' in user:
        sorted_traits = sorted(user['personality'].items(), key=lambda x: -x[1])
        best_traits = ', '.join([f"{k}: {round(v,2)}" for k,v in sorted_traits[:5]])
        lines.append(f"Top personality traits: {best_traits}")
    return "\n".join(lines)


In [2]:
def build_llm_prompt(new_user, db_users, top_k=5):
    text = f"You are a highly skilled psychologist specializing in human behavior and matchmaking. Your task is to match the query user with the most compatible individuals from the provided list of candidate profiles.\n\n"
    
    text += f"Below is the profile of a query user. Evaluate this profile based on personality traits, life priorities, interests, values, cognitive styles, and emotional compatibility, considering **how they would interact during a trip together**.\n\n"
    
    text += "Query User Profile:\n"
    text += profile_to_natural_text(new_user) + "\n\n"
    
    text += f"Next, you will be provided with a list of {len(db_users)} candidate profiles. For each candidate, evaluate their compatibility with the query user on a deeper psychological and emotional level. Consider the following:\n"
    text += "- **Shared interests**: Do they have overlapping hobbies or activities that would make the trip enjoyable?\n"
    text += "- **Life priorities**: Are their values, goals, and ambitions aligned, especially for shared experiences?\n"
    text += "- **Emotional compatibility**: Do their emotional responses complement each other, leading to a harmonious trip experience?\n"
    text += "- **Personality dynamics**: Not just introversion vs. extroversion—consider creativity, adaptability, empathy, and flexibility. How will these influence their interaction during travel?\n"
    text += "- **Learning and communication styles**: How do their thinking and problem-solving styles align, especially when navigating new environments during travel?\n"
    text += "- **Cultural and contextual factors**: How might their different backgrounds, countries, or locations influence their travel preferences and interaction?\n"
    text += "- **Conflict resolution and growth**: How well do they handle challenges or conflicts that might arise during travel? Do they complement or clash in their approach?\n"
    
    text += "\nFor each candidate, after considering all the above factors, provide the following:\n"
    text += "- **Name**: The candidate's name.\n"
    text += "- **Explanation**: A **short** explanation of why this candidate would be a good match for the query user, considering the above factors. Use a **personalized perspective**, for example: 'This match is ideal for you because you share the same love for traveling and photography.'\n"
    text += "- **Compatibility Score**: A score between 0 and 1, reflecting how well they align with the query user in a travel context.\n\n"
    
    text += "Format your answer as a JSON list of objects with the following keys: name, explanation, compatibility_score.\n"
    
    # Adding candidate profiles
    text += "\nCandidate Profiles:\n"
    for i, u in enumerate(db_users):
        text += f"\n[{i+1}] " + profile_to_natural_text(u)
        
    return text


In [ ]:
import openai
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access the API key from the environment variable
openai.api_key = os.getenv("API")

# Your function using the API
def llm_find_matches(new_user, db_users, top_k=5):
    prompt = build_llm_prompt(new_user, db_users, top_k)
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=1500
    )
    import re
    import ast
    import json
    llm_reply = response.choices[0].message['content']
    try:
        json_start = llm_reply.find('[')
        json_end = llm_reply.rfind(']')
        matches_json = llm_reply[json_start:json_end+1]
        matches = json.loads(matches_json)
    except Exception:
        matches = []
    return matches


In [4]:
# Prepare your new user and database
with open("travel_ready_user_profiles.json", "r", encoding="utf-8") as f:
    db_profiles = json.load(f)

user_data = {
    "name": "TestUser",
    "age": 28,
    "gender": "Male",
    "location": "Berlin",
    "country": "Germany",
    "education": "Master",
    "occupation": "Engineer",
    "marital_status": "Single",
    "learning_style": "Visual",
    "humor_style": "Witty",
    "interests": ["photography", "traveling", "chess", "reading"],
    "personality": {
        "extraversion": 0.7,
        "openness": 0.8,
        "agreeableness": 0.6,
        "conscientiousness": 0.5,
        "neuroticism": 0.3,
        "assertiveness": 0.6,
        "cheerfulness": 0.7,
        "imagination": 0.8,
        "intellect": 0.8,
        "artistic_interest": 0.7,
        "trust": 0.5,
        "altruism": 0.6,
        "cooperation": 0.7,
        "self_discipline": 0.7,
        "achievement_striving": 0.7,
        "cautiousness": 0.5,
        "adaptability": 0.7,
        "curiosity": 0.7,
        "creativity": 0.85,
        "risk_taking": 0.55,
        "empathy": 0.6,
        "optimism": 0.7,
        "motivation": 0.8,
        "introversion": 0.4,
        "ambition": 0.75
    },
    "languages": ["en", "de"],
    "bio": "I am passionate about exploring new places, love photography and a good chess game. Always ready for a new book or a travel adventure!"
}

# For efficiency, select the 20-50 most likely candidates first (optional, not required)
# For demo, just send all or top N to LLM
top_k = 5
candidate_profiles = db_profiles[:30]  # Adjust as you want

matches = llm_find_matches(user_data, candidate_profiles, top_k=top_k)
for match in matches:
    print(json.dumps(match, indent=2))


{
  "name": "Diana3",
  "explanation": "This match is ideal for you because you share the same love for traveling and photography. Diana3 also enjoys chess, which aligns with your interests, and their openness to new experiences complements your adventurous spirit.",
  "compatibility_score": 0.85
}
{
  "name": "Ben27",
  "explanation": "Ben27 shares your interests in photography, chess, and traveling, making them a great companion for a trip. Their curiosity and agreeableness align well with your personality traits, ensuring smooth communication and shared enthusiasm for exploration.",
  "compatibility_score": 0.82
}
{
  "name": "Mona12",
  "explanation": "Mona12 enjoys photography and chess, aligning well with your interests. Her creativity and openness to new experiences match your personality, promising engaging conversations and a shared appreciation for the arts during travel.",
  "compatibility_score": 0.8
}
{
  "name": "Jack9",
  "explanation": "Jack9 shares your love for photog